 Downloading the Data

In [1]:
import os
import tarfile
import urllib.request

Down_path_root = "http://spamassassin.apache.org/old/publiccorpus/"
spam_path = Down_path_root + "20030228_easy_ham.tar.bz2"
ham_path = Down_path_root + "20030228_spam.tar.bz2"
out_path = os.path.join("datasets","spam")

def fetch_data(ham_url = ham_path,spam_url = spam_path,output = out_path):
    if not os.path.isdir(output):
        os.makedirs(output)
    for filename,url in (("ham.tar.bz2",ham_url),("spam.tar.bz2",spam_url)):
        path = os.path.join(output,filename)
        if not os.path.isfile(path):
            urllib.request.urlretrieve(url,path)
        tar_bz2_file = tarfile.open(path)
        tar_bz2_file.extractall(path=output)
        tar_bz2_file.close()

In [2]:
fetch_data()

Loading the Data

In [4]:
hamDir = os.path.join(out_path,"easy_ham")
spamDir = os.path.join(out_path,"spam")
hamFileNames = [name for name in sorted(os.listdir(hamDir)) if len(name) > 20]
spamFileNames = [name for name in sorted(os.listdir(spamDir)) if len(name) > 20]

In [5]:
len(hamFileNames)

2500

In [6]:
len(spamFileNames)

500

using Python Email module for parsing

In [7]:
import email
import email.policy

def load_email(is_spam, filename, spamPath = out_path):
    directory = "spam" if is_spam else "easy_ham"
    with open(os.path.join(spamPath,directory,filename),"rb") as f:
        return email.parser.BytesParser(policy=email.policy.default).parse(f)

In [8]:
hamEmails = [load_email(is_spam = False, filename = name) for name in hamFileNames]
spamEmails = [load_email(is_spam = True, filename = name) for name in spamFileNames]

In [9]:
len(hamEmails)

2500

In [10]:
print(hamEmails[1].get_content().strip())

Martin A posted:
Tassos Papadopoulos, the Greek sculptor behind the plan, judged that the
 limestone of Mount Kerdylio, 70 miles east of Salonika and not far from the
 Mount Athos monastic community, was ideal for the patriotic sculpture. 
 
 As well as Alexander's granite features, 240 ft high and 170 ft wide, a
 museum, a restored amphitheatre and car park for admiring crowds are
planned
---------------------
So is this mountain limestone or granite?
If it's limestone, it'll weather pretty fast.

------------------------ Yahoo! Groups Sponsor ---------------------~-->
4 DVDs Free +s&p Join Now
http://us.click.yahoo.com/pt6YBB/NXiEAA/mG3HAA/7gSolB/TM
---------------------------------------------------------------------~->

To unsubscribe from this group, send an email to:
forteana-unsubscribe@egroups.com

 

Your use of Yahoo! Groups is subject to http://docs.yahoo.com/info/terms/


In [11]:
print(spamEmails[1].get_content().strip())

1) Fight The Risk of Cancer!
http://www.adclick.ws/p.cfm?o=315&s=pk007

2) Slim Down - Guaranteed to lose 10-12 lbs in 30 days
http://www.adclick.ws/p.cfm?o=249&s=pk007

3) Get the Child Support You Deserve - Free Legal Advice
http://www.adclick.ws/p.cfm?o=245&s=pk002

4) Join the Web's Fastest Growing Singles Community
http://www.adclick.ws/p.cfm?o=259&s=pk007

5) Start Your Private Photo Album Online!
http://www.adclick.ws/p.cfm?o=283&s=pk007

Have a Wonderful Day,
Offer Manager
PrizeMama













If you wish to leave this list please use the link below.
http://www.qves.com/trim/?ilug@linux.ie%7C17%7C114258


-- 
Irish Linux Users' Group: ilug@linux.ie
http://www.linux.ie/mailman/listinfo/ilug for (un)subscription information.
List maintainer: listmaster@linux.ie
